In [ ]:
import os
import sklearn
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras.callbacks as callbacks
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import pandas as pd
# Display
from IPython.display import Image, display
import matplotlib.cm as cm
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sn
import pydicom as dicom
import subprocess
import gc

size = 224

In [ ]:
class Dataset:
    def __init__(self):
        self.x_path = "../input/plant-seedlings-classification/train"
        self.classes = os.listdir(self.x_path)
        self.x_data = []
        self.y_data = []
        
    def __preprocessImage(self, image):
        image = np.array(image, dtype = "float32")
        image = cv2.resize(image, (size, size))
        image = np.reshape(image, (size, size, 3)) 
        image /= 255.0
        
        return np.array(image, dtype = "float32")
    
    def __extractImages(self, path):
        images = os.listdir(path)
        for image_path in images:
            image = cv2.imread(path + "/" + image_path)
            self.x_data.append(self.__preprocessImage(image))
            self.y_data.append(self.classes.index(path.split("/")[-1]))
            
    def load_data(self):
        i = 0
        for index in self.classes:
            i += 1
            self.__extractImages(self.x_path + "/" + index)
            print(f"{(i / len(self.classes)) * 100}% Data Loaded, Index = {i}", end = "\r")
        
        #self.x_data = np.reshape(self.x_data, (np.shape(self.x_data)[0], 1, size, size, 3))
        self.x_data = np.array(self.x_data, dtype = "float32")
        self.y_data = to_categorical(self.y_data)
        self.y_data = np.array(self.y_data, dtype = "float32")
        
        print("Data Successfully Loaded")
        return train_test_split(self.x_data, self.y_data, random_state = 42)
    
dataset = Dataset()
x_train, x_test, y_train, y_test = dataset.load_data()
y_train = tf.constant(y_train)
y_test = tf.constant(y_test)

del dataset
gc.collect()

In [ ]:
print(y_train.shape)
print(x_train.shape)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

def ImageGenerator():
    return ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest")

In [ ]:
class CNNLSTM:
    def __init__(self, shape, cnn):
        self.shape = shape
        self.model_input = keras.layers.Input(shape = (self.shape[0], self.shape[1], self.shape[2]), ragged = True, dtype = 'float32')
        self.cnn = cnn(weights = "imagenet", include_top = False, input_tensor = self.model_input)
        for layer in self.cnn.layers[: len(self.cnn.layers) // 2 ]:
            layer.trainable = True
        for layer in self.cnn.layers[len(self.cnn.layers) // 2 :]:
            layer.trainable = False
        self.model = 0
        
        self.save_path = "./cnnlstm.h5"
        self.__build_model()

    def __build_model(self):
        
        #print(self.cnn.output.shape)
        x = keras.layers.Flatten()(self.cnn.output)
        x = keras.layers.Dense(1024, activation ="selu", kernel_initializer = "lecun_normal")(x)
        x = keras.layers.Dropout(0.1)(x)
        x = keras.layers.Dense(512, activation = "selu", kernel_initializer = "lecun_normal")(x)
        x = keras.layers.Dropout(0.1)(x)
        x = keras.layers.Dense(256, activation = "selu", kernel_initializer = "lecun_normal")(x)
        x = keras.layers.Dropout(0.1)(x)
        x = keras.layers.Dense(128, activation = "selu", kernel_initializer = "lecun_normal")(x)
        x = keras.layers.Dropout(0.1)(x)
        x = keras.layers.Dense(64, activation =  "selu", kernel_initializer = "lecun_normal")(x)
        x = keras.layers.Dropout(0.1)(x)
        x = keras.layers.Dense(32, activation =  "selu", kernel_initializer = "lecun_normal")(x)
        x = keras.layers.Dropout(0.1)(x)
        
        output = keras.layers.Dense(12, activation = "softmax")(x)
        
        self.model = keras.models.Model(inputs = self.model_input,
                                  outputs = output, name = "CNNLSTM")
        #display(keras.utils.plot_model(self.model, show_shapes=True, show_layer_names=True))
        
    def plot_loss_acc(self):
        #Loss vs Epochs
        plt.plot(self.model.history.history['loss'], label='train')
        plt.plot(self.model.history.history['val_loss'], label='test')
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.title(f"{self.name} Loss vs Epochs")
        plt.legend()
        plt.show()
        #Accuracy vs Epochs
        plt.plot(self.model.history.history['accuracy'], label='train')
        plt.plot(self.model.history.history['val_accuracy'], label='test')
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")
        plt.title(f"{self.name} ACCURACY vs Epochs")
        plt.legend()
        plt.show()
        
    def scoring(self):
        y_pred = self.model.predict(x_test)
        #y_pred = self.model.predict(x_test)
        cm = confusion_matrix(y_test, y_pred)
        print(classification_report(y_test,y_pred))
        df_cm = pd.DataFrame(cm)
        plt.figure(figsize = (10,7))
        sn.heatmap(df_cm, annot=True)
        
    def compile_and_run(self, lr = 1e-4, epochs = 100):
        optimizer = keras.optimizers.Adam(learning_rate = lr)
        checkpoint = keras.callbacks.ModelCheckpoint(self.save_path, save_best_only = True)
        self.model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
        self.model.fit(x = ImageGenerator().flow(x_train,y_train), epochs = epochs,
                       validation_data = (x_test, y_test),shuffle=True, workers = -1,
                       batch_size = 32, callbacks = [checkpoint])
        #self.scoring()
        #self.plot_loss_acc()

In [ ]:
cnnlstm = CNNLSTM((size, size, 3), keras.applications.resnet.ResNet101)
cnnlstm.compile_and_run()

In [ ]:
def __preprocessImage(image):
    image = np.array(image, dtype = "float32")
    image = cv2.resize(image, (size, size))
    image = np.reshape(image, (size, size, 3)) 
    image /= 255.0
    
    return np.array(image, dtype = "float32")

x_data_test = []
    
def __extractImages(path):
    global x_data_test
    image = cv2.imread(path)
    x_data_test.append(__preprocessImage(image))

x_path = "../input/plant-seedlings-classification/test"
images = os.listdir(x_path)

for image in images:
    __extractImages(x_path + "/" + image)
x_data_test = np.array(x_data_test, dtype = "float32")

In [ ]:
classes = os.listdir("../input/plant-seedlings-classification/train")
predictions = cnnlstm.model.predict(x_data_test)
df = pd.DataFrame()
df['file'] = images
df['species'] = [classes[np.argmax(pred)] for pred in predictions]
df.index = df['file']
df = df.drop('file', axis = 1)
df.to_csv("./submission.csv")